In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import accuracy_score
import joblib

In [4]:
def feature_selection(X_train, y_train):
    # Identify categorical features
    categorical_features = X_train.select_dtypes(include=['object']).columns.tolist()

    # Create a preprocessor that includes one-hot encoding for categorical features
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', OneHotEncoder(), categorical_features)
        ],
        remainder='passthrough'  # Keep the numerical columns as they are
    )

    # Fit and transform the training data
    X_train_transformed = preprocessor.fit_transform(X_train)

    # Feature selection using SelectKBest
    selector = SelectKBest(score_func=f_classif, k='all')  # Use 'all' or adjust 'k' as needed
    X_train_selected = selector.fit_transform(X_train_transformed, y_train)

    return X_train_selected, selector, preprocessor


In [5]:
def train_model(X_train, y_train):
    # Initialize the RandomForestClassifier
    model = RandomForestClassifier(random_state=42)

    # Set up hyperparameters for tuning
    param_grid = {
        'n_estimators': [100, 200, 300],  # Number of trees
        'max_depth': [10, 20, 30],        # Maximum depth of the trees
        'min_samples_split': [2, 5, 10],  # Minimum samples required to split a node
        'min_samples_leaf': [1, 2, 4],    # Minimum samples required to be at a leaf node
        'bootstrap': [True, False]        # Whether to use bootstrap samples
    }

    # Perform GridSearchCV to find the best hyperparameters
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
    grid_search.fit(X_train, y_train)

    # Return the best model found by GridSearchCV
    return grid_search.best_estimator_


In [ ]:
def main():
    # Load your dataset
    df = pd.read_csv("dataset.csv")  

    # Prepare features (X) and target (y)
    X = df.drop('Preferred_Product_Type', axis=1)
    y = df['Preferred_Product_Type']

    # Encode target variable as numeric values
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
    )

    # Perform feature selection
    X_train_selected, selector, preprocessor = feature_selection(X_train, y_train)

    # Train model
    model = train_model(X_train_selected, y_train)

    # Preprocess and select features for the test set
    X_test_transformed = preprocessor.transform(X_test)
    X_test_selected = selector.transform(X_test_transformed)

    # Make predictions
    y_pred = model.predict(X_test_selected)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Model Accuracy: {accuracy:.2f}")

    # Save the trained model and preprocessor
    model_filename = 'random_forest_model.pkl'
    joblib.dump(model, model_filename)
    print(f"Model saved to {model_filename}")

    preprocessor_filename = 'preprocessor.pkl'
    joblib.dump(preprocessor, preprocessor_filename)
    print(f"Preprocessor saved to {preprocessor_filename}")


In [7]:
if __name__ == "__main__":
    main()


Fitting 3 folds for each of 162 candidates, totalling 486 fits


C:\Users\Jainivas Anandhan\anaconda3\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Model Accuracy: 0.27
Model saved to random_forest_model.pkl
Preprocessor saved to preprocessor.pkl
